In [25]:
import nltk
import string
import re,os
from nltk.stem import PorterStemmer

In [26]:
stop_words=[]
ps = PorterStemmer()
with open('Stopword-List.txt','r') as f:
    stop_words = f.read().split()

print(stop_words)
for i,v in enumerate(stop_words):
    stop_words[i] = ps.stem(v)



['a', 'is', 'the', 'of', 'all', 'and', 'to', 'can', 'be', 'as', 'once', 'for', 'at', 'am', 'are', 'has', 'have', 'had', 'up', 'his', 'her', 'in', 'on', 'no', 'we', 'do']


In [27]:
re1=r"[a-z]+((?:-[a-z]+)*)?"
print(re.search(re1,"\"how\"").group())
print(ps.stem('ai'))


how
ai


In [28]:
def isConsecChar(val):
    count = 0
    max_count=0
    for i,v in enumerate(val):
        if i==len(val)-1 and count>0:
            count+=1
            continue
        elif i<len(val)-1:
            if v==val[i+1]:
                count+=1
            else:
                count=0
        if count>max_count:
            max_count = count
        # print(f'Index: {i} Char: {v} count: {count} max_c: {max_count}')
    return max_count+1

val = "bbbbbbbbccccccdeeee"


print(isConsecChar(val))



8


In [39]:
inverted_index = {}
docs=[]
count=0
re1=r'[a-z]+((?:-[a-z]+)*)?'
filenames=os.listdir('ResearchPapers/')
for i in filenames:
    doc_id=int(i.strip('.txt'))
    docs.append(doc_id)
    file = os.path.join('ResearchPapers',i)
    with open(file,'r',encoding='utf-8', errors='ignore') as f:
        text = f.read()
        text=text.lower().split()
        count+=len(text)
        for index, val in enumerate(text):
            if "@" not in val and ".co" not in val and "http" not in val and isConsecChar(val)<3:
                match = re.search(re1,val)
                if match:
                    val = match.group()
                    if "-" in val and val.count('-')==1:
                        val = val.replace("-","")
                    val= ps.stem(val)
                    if (val not in stop_words) and (len(val)>1) and (len(val)<16):
                        if val in inverted_index:
                            if doc_id in inverted_index[val]:
                                inverted_index[val][doc_id].append(index)
                            else:
                                inverted_index[val][doc_id] = [index]
                        else:
                            inverted_index[val] = {doc_id: [index]}




print("Total Tokkens: ",count)
print("Number of terms:",len(inverted_index))
print("Tokkens drops: ",((count-len(inverted_index))/count)*100,"%")
print(inverted_index[ps.stem('overview')])







Total Tokkens:  196879
Number of terms: 13357
Tokkens drops:  93.21562990466225 %
{1: [0, 798, 872, 902, 958, 5903, 8769, 10294, 10439], 12: [498, 5758, 8361], 15: [1166, 1266, 4503, 4670], 16: [774, 842, 1036, 6311, 6944, 8233, 8575], 17: [7628], 2: [3019, 4840, 7767], 22: [65], 7: [225, 1280, 1402, 1827, 3499, 8488, 17512, 18618, 21859, 22513, 24716, 27705, 28052, 28284, 28585, 36388, 38967]}


In [30]:
def intersectTerms(term1,term2):
    inverted_index_1 = inverted_index[ps.stem(term1)]
    inverted_index_2 = inverted_index[ps.stem(term2)]
    doc_list=[]
    if len(inverted_index_1)<len(inverted_index_2):
        for i in inverted_index_1.keys():
            if i in inverted_index_2.keys():
                doc_list.append(i)
    else:
        for i in inverted_index_2.keys():
            if i in inverted_index_1.keys():
                doc_list.append(i)
    return doc_list
        
    

In [31]:
def intersectPTerms(term1,term2,k):
    inverted_index_1 = inverted_index[ps.stem(term1)]
    inverted_index_2 = inverted_index[ps.stem(term2)]
    doc_list=set()
    if len(inverted_index_1)<len(inverted_index_2):
        for i in inverted_index_1.keys():
            if i in inverted_index_2.keys():
                for x in inverted_index_1[i]:
                    for y in inverted_index_2[i]:
                        if abs(y-x)<=k:
                            doc_list.add(i)
    else:
        for i in inverted_index_2.keys():
            if i in inverted_index_1.keys():
                for x in inverted_index_2[i]:
                    for y in inverted_index_1[i]:
                        if abs(y-x)<=k:
                            doc_list.add(i)
    return sorted(list(doc_list))

In [32]:
def unionTerms(term1,term2):
    inverted_index_1 = set(inverted_index[ps.stem(term1)].keys())
    inverted_index_2 = set(inverted_index[ps.stem(term2)].keys())
    inverted_index_1.union(inverted_index_2)
    return sorted(list(inverted_index_1))
    

In [33]:
def inverseTerms(term):
    inverted_index_set = set(inverted_index[ps.stem(term)].keys())
    return list((set(docs)).difference(inverted_index_set))

In [34]:
print(inverted_index[ps.stem('transformer')].keys())
print(inverseTerms('transformer'))
print(docs)


dict_keys([12, 16, 17, 18, 2, 21, 22, 24, 26, 7, 8])
[1, 3, 9, 11, 13, 14, 15, 23, 25]
[1, 2, 3, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26]


In [66]:
user_input = (input("Enter query: ")).lower()
intersection_list=[]
inter_lists=[]
union_list=[]
inverse_list=[]
inv_list=[]
result = set()
if "and" in user_input: 
    inter_list = user_input.split(" and ") 
    for x in inter_list: 
        if "or" in x: 
            unions = x.split(" or ")
            for y in unions:
                if "not" in y:
                    inverse_term = y.replace("not ",'').strip()
                    inv_list = inverseTerms(inverse_term)
                else:
                    inverse_term = y.strip()
                    inv_list = list(inverted_index[ps.stem(inverse_term)].keys())
                for i in inv_list:
                    union_list.append(i)
        else: 
            if "not" in x: 
                inverse_term = x.replace("not ",'').strip()
                inv_list = inverseTerms(inverse_term)
            else: 
                inverse_term = x.strip() 
                inv_list = list(inverted_index[ps.stem(inverse_term)].keys()) 
            for i in inv_list:
                union_list.append(i) 
        intersection_list.append(union_list) 
        union_list=[] 
    result=set(intersection_list[0]) 
    for x in intersection_list:
        result = result.intersection(set(x)) 
    # print(intersection_list)
    
elif "or" in user_input:
    unions = user_input.split(" or ")
    for y in unions:
        if "not" in y:
            inverse_term = y.replace("not ",'').strip()
            inv_list = inverseTerms(inverse_term)
        else:
            inverse_term = y.strip()
            inv_list = list(inverted_index[ps.stem(inverse_term)].keys())
        for i in inv_list:
            union_list.append(i)
    result = set(union_list)

elif "not" in user_input:
    inverse_term = user_input.replace("not ",'').strip()
    result = inverseTerms(inverse_term)
    
elif "/" in user_input:
    temp_list = user_input.split(" / ")
    print(temp_list)
    terms = temp_list[0].split()
    k = int(temp_list[1])
    result = intersectPTerms(terms[0],terms[1],k)
    
    
print(sorted(list(result)))
            
                    


['overview learning', '33']
[1, 2, 7, 15, 16, 17]


In [69]:
# NOT cancer AND NOT overview OR NOT heart
# print(intersectTerms('heart','cancer'))
print("not tdd".replace("not ",''))
print(inverseTerms('heart'))
print(inverseTerms("overview"))
# print(inverted_index[ps.stem('heart')].keys())
print(inverted_index[ps.stem('overview')][17])
print(inverted_index[ps.stem('learning')][17])
print(ps.stem('learning'))

# print(inverseTerms('overview'))
# print(inverseTerms('cancer'))
# print(inverseTerms('heart'))
# print(sorted_inverted_index[ps.stem('heart')].keys())



tdd
[2, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25]
[3, 8, 9, 11, 13, 14, 18, 21, 23, 24, 25, 26]
[7628]
[101, 364, 546, 815, 1492, 2165, 2304, 2764, 3753, 3883, 4372, 4487, 4583, 4586, 4589, 4592, 4600, 4709, 4720, 4780, 4784, 4790, 4864, 4880, 4905, 4927, 4938, 5003, 5006, 5014, 5038, 5049, 5057, 5186, 5458, 5568, 5591, 5623, 5651, 5692, 5752, 5875, 5926, 5935, 5979, 6004, 6030, 6039, 6075, 6110, 6120, 6129, 6154, 6231, 6237, 6257, 6275, 6301, 6340, 6364, 6376, 6412, 6446, 6500, 6510, 6614, 6693, 6731, 6827, 7404, 7602, 7769, 7917, 7955, 7979]
learn
